In [ ]:
!pip install yt_dlp transformers datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 60.7 MB/s eta 0:00:00:00:01


In [ ]:
import requests
import yt_dlp
import tempfile
import shutil
import os
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

In [ ]:
def download_youtube_audio(youtube_url):
    temp_dir = tempfile.mkdtemp()
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': os.path.join(temp_dir, 'audio.%(ext)s'),
        'quiet': True,
        'no_warnings': True,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(youtube_url, download=True)
            audio_file = os.path.join(temp_dir, "audio.mp3")  # Ensure correct extension
            return audio_file
    except Exception as e:
        print(f"Error downloading audio: {e}")
        return None


In [ ]:
audio = download_youtube_audio("https://youtu.be/Fbg7ChsjmEA?si=ANh-EYzZty81-Jhx")

In [ ]:
if audio:
    target_path = os.path.join(os.getcwd(), "audio.mp3")
    shutil.move(audio, target_path)
    print(f"Audio saved to: {target_path}")
else:
    print("Failed to download audio.")

Audio saved to: /kaggle/working/audio.mp3


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [ ]:
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(torch_dtype)

torch.float16


In [ ]:
model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bia

In [ ]:
processor = AutoProcessor.from_pretrained(model_id)


preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [ ]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

Device set to use cuda:0


In [ ]:
dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")

README.md:   0%|          | 0.00/480 [00:00<?, ?B/s]

(…)-00000-of-00001-913508124a40cb97.parquet:   0%|          | 0.00/1.98M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
result = pipe(target_path,return_timestamps=True)
print(result["text"])
transcript = result["text"]

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


 Here it is, NVIDIA's new RTX 5080. Compared to its predecessor, the 4080 Super, it is around 15% faster in games that use its new features. It's faster in AI, as long as you don't need more than 16 gigs of RAM for your model. It's more efficient, even though it draws more power overall. And it's got a great cooler design, as long as you don't mind heating up your CPU. And that's it. That's the most positive spin we can put on this thing. Why the do we need all of these caveats when we say the new one is better than the old one? I mean, isn't the point of the new one to be clearly and substantially better? Well, it is better. In cranked out Cyberpunk at 4K, it not only beats the 4080 Super, but it even gives the 4090 a run for its money. But it's such a mixed bag that in other situations, it barely outcompetes the 7900 XTX, the flagship GPU that made AMD think, You know, maybe we just shouldn't do flagships anymore. Hey, glass half full? At least they didn't raise the price. And we did

In [ ]:
from transformers import pipeline

def process_transcript(transcript_text, target_lang="en"):
    print("\nSummarizing transcript in English...")
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    try:
        summary_english = summarizer(transcript_text, max_length=150, min_length=40, do_sample=False)[0]['summary_text']
    except Exception as e:
        print(f"Error during summarization: {e}")
        return


    if target_lang != "en":
        print(f"\nTranslating summary to {target_lang}...")
        model_name = f"Helsinki-NLP/opus-mt-en-{target_lang}"
        try:
            translator = pipeline("translation", model=model_name)
            translation_result = translator(summary_english)
            summary_final = translation_result[0]['translation_text']
        except Exception as e:
            print(f"Error during translation: {e}")
            return
    else:
        summary_final = summary_english

    print("\n--- Summary ---")
    print(summary_final)



In [ ]:
process_transcript(transcript, target_lang="es")